# Agregação de dados e operações em grupos

## Funcionamento de GroupBy

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({'key1':['a','a','b','b','a'],'key2':['one','two','one','two','one'],'data1':np.random.randn(5),'data2':np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,0.110936,1.292879
1,a,two,0.269065,1.739307
2,b,one,-1.131447,-1.720543
3,b,two,0.545836,-1.820084
4,a,one,-0.877297,-0.790506


In [3]:
grouped = df['data1'].groupby(df['key1'])

grouped

In [4]:
grouped.mean()

key1
a   -0.165765
b   -0.292806
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one    -0.383180
      two     0.269065
b     one    -1.131447
      two     0.545836
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,-0.383180,0.269065
b,-1.131447,0.545836


In [7]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])

years = np.array([2005,2005,2006,2005,2006])

df['data1'].groupby([states,years]).mean()

California  2005    0.269065
            2006   -1.131447
Ohio        2005    0.328386
            2006   -0.877297
Name: data1, dtype: float64

In [8]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.165765,0.747227
b,-0.292806,-1.770314


In [9]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.383180  0.251186
     two   0.269065  1.739307
b    one  -1.131447 -1.720543
     two   0.545836 -1.820084

In [10]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterando por grupos

In [11]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.110936  1.292879
1    a  two  0.269065  1.739307
4    a  one -0.877297 -0.790506
b
  key1 key2     data1     data2
2    b  one -1.131447 -1.720543
3    b  two  0.545836 -1.820084


In [12]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.110936  1.292879
4    a  one -0.877297 -0.790506
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.269065  1.739307
('b', 'one')
  key1 key2     data1     data2
2    b  one -1.131447 -1.720543
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.545836 -1.820084


In [14]:
pieces = dict(list(df.groupby('key1')))

pieces['b']

,key1,key2,data1,data2
2,b,one,-1.131447,-1.720543
3,b,two,0.545836,-1.820084


In [15]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [16]:
grouped = df.groupby(df.dtypes, axis=1)

grouped

In [17]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.110936  1.292879
1  0.269065  1.739307
2 -1.131447 -1.720543
3  0.545836 -1.820084
4 -0.877297 -0.790506
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecionando uma coluna ou um subconjunto de colunas

In [18]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.251186
     two   1.739307
b    one  -1.720543
     two  -1.820084

In [19]:
s_grouped = df.groupby(['key1','key2'])['data2']

s_grouped

In [20]:
s_grouped.mean()

key1  key2
a     one     0.251186
      two     1.739307
b     one    -1.720543
      two    -1.820084
Name: data2, dtype: float64

### Agrupando com dicionários e séries

In [21]:
people = pd.DataFrame(np.random.randn(5,5), columns=['a','b','c','d','e'], index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3,[1,2]] = np.nan

people

,a,b,c,d,e
Joe,-0.279233,-0.160279,0.451348,-0.656067,-0.280396
Steve,-1.296191,1.026409,0.661630,1.186656,-0.706273
Wes,0.867128,NaN,NaN,-0.011430,2.260382
Jim,0.512211,-0.021967,0.562008,-0.789647,0.004279
Travis,-0.262828,-0.865091,0.309476,0.982185,-0.601105


In [22]:
mapping = {
    'a': 'red',
    'b': 'red',
    'c': 'blue',
    'd': 'blue',
    'e': 'red',
    'f': 'orange'
}

by_column = people.groupby(mapping, axis=1)

by_column.sum()

,blue,red
Joe,-0.204719,-0.719907
Steve,1.848286,-0.976055
Wes,-0.011430,3.127510
Jim,-0.227638,0.494523
Travis,1.291661,-1.729023


In [23]:
map_series = pd.Series(mapping)

map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [24]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Agrupando com funções

In [25]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.100105,-0.182246,1.013356,-1.457143,1.984266
5,-1.296191,1.026409,0.661630,1.186656,-0.706273
6,-0.262828,-0.865091,0.309476,0.982185,-0.601105


In [26]:
key_list = ['one','one','one','two','two']

people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -0.279233 -0.160279  0.451348 -0.656067 -0.280396
  two  0.512211 -0.021967  0.562008 -0.789647  0.004279
5 one -1.296191  1.026409  0.661630  1.186656 -0.706273
6 two -0.262828 -0.865091  0.309476  0.982185 -0.601105

### Agrupando por níveis de índice

In [27]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]],names=['cty','tenor'])

hier_df = pd.DataFrame(np.random.randn(4,5),columns=columns)

hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.603960  1.113847 -0.155989 -0.162834  1.164767
1      0.024617 -0.010205  0.257639 -0.301814  1.871494
2      0.536362 -0.234872  1.468009  1.568661 -0.292467
3      1.457061  0.888422  0.170456  1.071589  1.391395

In [28]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Agregação de dados

In [29]:
df

,key1,key2,data1,data2
0,a,one,0.110936,1.292879
1,a,two,0.269065,1.739307
2,b,one,-1.131447,-1.720543
3,b,two,0.545836,-1.820084
4,a,one,-0.877297,-0.790506


In [30]:
grouped = df.groupby(['key1'])

grouped['data1'].quantile(0.9)

key1
a    0.237439
b    0.378108
Name: data1, dtype: float64

In [31]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

/tmp/ipykernel_20509/1321513124.py:4: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak)


,data1,data2
key1,,
a,1.146361,2.529813
b,1.677284,0.099541


In [32]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.165765  0.621256 -0.877297 -0.383180  0.110936  0.190000   
b      2.0 -0.292806  1.186019 -1.131447 -0.712127 -0.292806  0.126515   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.269065   3.0  0.747227  1.350293 -0.790506  0.251186  1.292879   
b     0.545836   2.0 -1.770314  0.070386 -1.820084 -1.795199 -1.770314   

                          
           75%       max  
key1                      
a     1.516093  1.739307  
b    -1.745428 -1.720543

### Aplicação de funções nas colunas e aplicação de várias funções

In [33]:
tips = pd.read_csv('pydata-book/examples/tips.csv')

# porcentagem de gorjeta sobre o total da conta
tips['tip_pct'] = tips['tip'] / tips['total_bill']

tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [34]:
grouped = tips.groupby(['day', 'smoker'])

grouped_pct = grouped['tip_pct']

grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [36]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [37]:
grouped_pct.agg([('foo','mean'),('bar',np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [38]:
functions = ['count','mean','max']

result = grouped['tip_pct','total_bill'].agg(functions)

result

/tmp/ipykernel_20509/3510728767.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct','total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [39]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [40]:
ftuples = [('Media','mean'),('Variancia',np.var)]

grouped['tip_pct','total_bill'].agg(ftuples)


/tmp/ipykernel_20509/1719379085.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct','total_bill'].agg(ftuples)


tip_pct           total_bill            
                Media Variancia      Media   Variancia
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [41]:
grouped.agg({'tip': np.max, 'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [43]:
grouped.agg({'tip': functions, 'size': 'sum'})

tip                  size
            count      mean    max  sum
day  smoker                            
Fri  No         4  2.812500   3.50    9
     Yes       15  2.714000   4.73   31
Sat  No        45  3.102889   9.00  115
     Yes       42  2.875476  10.00  104
Sun  No        57  3.167895   6.00  167
     Yes       19  3.516842   6.50   49
Thur No        45  2.673778   6.70  112
     Yes       17  3.030000   5.00   40

### Devolvendo dados agregados sem índices de linhas

In [44]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Método apply: separar-aplicar-combinar genérico

In [45]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [46]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [47]:
tips.groupby(['smoker','day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [49]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [51]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [52]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

In [56]:
grouped.describe()

total_bill                                                        \
                 count       mean        std    min     25%     50%      75%   
day  smoker                                                                    
Fri  No            4.0  18.420000   5.059282  12.46  15.100  19.235  22.5550   
     Yes          15.0  16.813333   9.086388   5.75  11.690  13.420  18.6650   
Sat  No           45.0  19.661778   8.939181   7.25  14.730  17.820  20.6500   
     Yes          42.0  21.276667  10.069138   3.07  13.405  20.390  26.7925   
Sun  No           57.0  20.506667   8.130189   8.77  14.780  18.430  25.0000   
     Yes          19.0  24.120000  10.442511   7.25  17.165  23.100  32.3750   
Thur No           45.0  17.113111   7.721728   7.51  11.690  15.950  20.2700   
     Yes          17.0  19.190588   8.355149  10.34  13.510  16.470  19.8100   

                     tip            ...  size      tip_pct            \
               max count      mean  ...   75%  max   count      mean   
day  smoker                         ...                                
Fri  No      22.75   4.0  2.812500  ...  2.25  3.0     4.0  0.151650   
     Yes     40.17  15.0  2.714000  ...  2.00  4.0    15.0  0.174783   
Sat  No      48.33  45.0  3.102889  ...  3.00  4.0    45.0  0.158048   
     Yes     50.81  42.0  2.875476  ...  3.00  5.0    42.0  0.147906   
Sun  No      48.17  57.0  3.167895  ...  4.00  6.0    57.0  0.160113   
     Yes     45.35  19.0  3.516842  ...  3.00  5.0    19.0  0.187250   
Thur No      41.19  45.0  2.673778  ...  2.00  6.0    45.0  0.160298   
     Yes     43.11  17.0  3.030000  ...  2.00  4.0    17.0  0.163863   

                                                                         
                  std       min       25%       50%       75%       max  
day  smoker                                                              
Fri  No      0.028123  0.120385  0.137239  0.149241  0.163652  0.187735  
     Yes     0.051293  0.103555  0.133739  0.173913  0.209240  0.263480  
Sat  No      0.039767  0.056797  0.136240  0.150152  0.183915  0.291990  
     Yes     0.061375  0.035638  0.091797  0.153624  0.190502  0.325733  
Sun  No      0.042347  0.059447  0.139780  0.161665  0.185185  0.252672  
     Yes     0.154134  0.065660  0.097723  0.138122  0.215325  0.710345  
Thur No      0.038774  0.072961  0.137741  0.153492  0.184843  0.266312  
     Yes     0.039389  0.090014  0.148038  0.153846  0.194837  0.241255  

[8 rows x 32 columns]

### Suprimindo as chaves de grupo

In [57]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Análise de quantis e de buckets

In [58]:
frame = pd.DataFrame({'data1':np.random.randn(1000),'data2': np.random.randn(1000)})

quartiles = pd.cut(frame.data1,4)

quartiles[:10]

0     (-0.545, 1.295]
1     (-0.545, 1.295]
2     (-0.545, 1.295]
3     (-0.545, 1.295]
4      (1.295, 3.136]
5     (-0.545, 1.295]
6      (1.295, 3.136]
7    (-2.386, -0.545]
8    (-2.386, -0.545]
9     (-0.545, 1.295]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-4.234, -2.386] < (-2.386, -0.545] < (-0.545, 1.295] < (1.295, 3.136]]

In [59]:
def get_stats(group):
    return {
        'min': group.min(),
        'max': group.max(),
        'count': group.count(),
        'mean' : group.mean()
    }


grouped = frame.data2.groupby(quartiles)

grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-4.234, -2.386]",-2.034743,2.179258,5.0,-0.023978
"(-2.386, -0.545]",-2.715208,3.368534,297.0,0.046432
"(-0.545, 1.295]",-3.107808,2.919307,603.0,-0.016364
"(1.295, 3.136]",-2.130179,1.944935,95.0,-0.094807


In [60]:
# Devolve os números dos quantis
grouping = pd.qcut(frame.data1, 10, labels=False)

grouped = frame.data2.groupby(grouping)

grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.473244,2.179258,100.0,0.022613
1,-2.038845,3.368534,100.0,0.062391
2,-2.715208,2.275300,100.0,0.054461
3,-2.298050,2.919307,100.0,-0.078000
4,-2.637417,2.754317,100.0,0.080335
5,-3.107808,2.676221,100.0,-0.036524
6,-2.187255,2.249339,100.0,-0.080642
7,-2.145835,2.043330,100.0,-0.037471
8,-2.758084,2.517447,100.0,-0.000424


### Preenchendo valores ausentes com valores específicos de grupo

In [61]:
s = pd.Series(np.random.randn(6))

s[::2] = np.nan

s

0         NaN
1   -0.637014
2         NaN
3    0.436657
4         NaN
5   -0.111389
dtype: float64

In [62]:
s.fillna(s.mean())

0   -0.103915
1   -0.637014
2   -0.103915
3    0.436657
4   -0.103915
5   -0.111389
dtype: float64

In [63]:
states = ['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']

grouped_key = ['East'] * 4 + ['West'] * 4

data = pd.Series(np.random.randn(8),index=states)

data

Ohio         -1.607963
New York     -0.160991
Vermont       0.050703
Florida       1.292305
Oregon        1.944184
Nevada       -0.099615
California   -1.039495
Idaho         0.678806
dtype: float64

In [64]:
data[['Vermont','Nevada','Idaho']] = np.nan

data

In [65]:
data.groupby(grouped_key).mean()

East   -0.158883
West    0.452344
dtype: float64

In [66]:
fill_mean = lambda g: g.fillna(g.mean())

data.groupby(grouped_key).apply(fill_mean)

Ohio         -1.607963
New York     -0.160991
Vermont      -0.158883
Florida       1.292305
Oregon        1.944184
Nevada        0.452344
California   -1.039495
Idaho         0.452344
dtype: float64

In [69]:
fill_values = {'East':0.5,'West':-1}

fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(grouped_key).apply(fill_func)

Ohio         -1.607963
New York     -0.160991
Vermont       0.500000
Florida       1.292305
Oregon        1.944184
Nevada       -1.000000
California   -1.039495
Idaho        -1.000000
dtype: float64

### Amostragem aleatória e permutação

In [70]:
# Copas (Hearts), Espada (Spades), Paus (Clubs), Ouro (Diamonds)
suits = ['H','S','C','D']

card_val = (list(range(1,11)) + [10] * 3) * 4

base_names = ['A'] + list(range(2,11)) + ['J','K','Q']

cards = []

for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [72]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

10D    10
5D      5
7D      7
8H      8
9S      9
dtype: int64

In [73]:
get_suit = lambda card: card[-1] # última letra é o naipe

deck.groupby(get_suit).apply(draw , n=2)

C  QC     10
   4C      4
D  3D      3
   QD     10
H  3H      3
   5H      5
S  10S    10
   2S      2
dtype: int64

In [74]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

5C      5
8C      8
KD     10
6D      6
2H      2
3H      3
10S    10
JS     10
dtype: int64

### Média ponderada de grupos e correlação

In [75]:
df = pd.DataFrame({'category':['a','a','a','a','b','b','b','b'],'data':np.random.randn(8),'weights':np.random.rand(8)})

df

,category,data,weights
0,a,0.325333,0.089262
1,a,-0.166777,0.862554
2,a,0.288445,0.779348
3,a,-2.328346,0.663194
4,b,0.224413,0.477152
5,b,-1.132194,0.316836
6,b,0.311276,0.070716
7,b,0.450204,0.179141


In [77]:
grouped = df.groupby('category')

get_wavg = lambda g: np.average(g['data'],weights=g['weights'])

grouped.apply(get_wavg)

category
a   -0.598975
b   -0.142721
dtype: float64

In [78]:
close_px = pd.read_csv('pydata-book/examples/stock_px_2.csv', parse_dates=True,index_col=0)

close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [79]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [80]:
spx_corr = lambda x: x.corrwith(x['SPX'])

rets = close_px.pct_change().dropna()

get_year = lambda x: x.year

by_year = rets.groupby(get_year)

by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [82]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Regressão linear nos grupos

In [83]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y,X).fit()
    return result.params

In [84]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## Tabelas pivôs e tabulação cruzada

In [85]:
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [86]:
tips.pivot_table(['tip_pct','size'], index=['time','day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [87]:
tips.pivot_table(['tip_pct','size'], index=['time','day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [88]:
tips.pivot_table(['tip_pct','size'], index=['time','day'], columns='smoker', margins=True, aggfunc=len)

size            tip_pct           
smoker          No   Yes  All      No   Yes  All
time   day                                      
Dinner Fri     3.0   9.0   12     3.0   9.0   12
       Sat    45.0  42.0   87    45.0  42.0   87
       Sun    57.0  19.0   76    57.0  19.0   76
       Thur    1.0   NaN    1     1.0   NaN    1
Lunch  Fri     1.0   6.0    7     1.0   6.0    7
       Thur   44.0  17.0   61    44.0  17.0   61
All          151.0  93.0  244   151.0  93.0  244

In [90]:
tips.pivot_table('tip_pct', index=['time','size', 'smoker'], columns='day', margins=True, aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur       All
time   size smoker                                                  
Dinner 1    No      0.000000  0.137931  0.000000  0.000000  0.137931
            Yes     0.000000  0.325733  0.000000  0.000000  0.325733
       2    No      0.139622  0.162705  0.168859  0.159744  0.164383
            Yes     0.171297  0.148668  0.207893  0.000000  0.167246
       3    No      0.000000  0.154661  0.152663  0.000000  0.153705
            Yes     0.000000  0.144995  0.152660  0.000000  0.148061
       4    No      0.000000  0.150096  0.148143  0.000000  0.148737
            Yes     0.117750  0.124515  0.193370  0.000000  0.139064
       5    No      0.000000  0.000000  0.206928  0.000000  0.206928
            Yes     0.000000  0.106572  0.065660  0.000000  0.086116
       6    No      0.000000  0.000000  0.103799  0.000000  0.103799
Lunch  1    No      0.000000  0.000000  0.000000  0.181728  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000  0.223776
       2    No      0.000000  0.000000  0.000000  0.166005  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843  0.165266
       3    No      0.187735  0.000000  0.000000  0.084246  0.118742
            Yes     0.000000  0.000000  0.000000  0.204952  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706  0.173706
All                 0.169913  0.153152  0.166897  0.161276  0.160803

In [92]:
from io import StringIO

data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""

data = pd.read_table(StringIO(data), sep='\s+')

data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [93]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [94]:
pd.crosstab([tips.time,tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244